<a href="https://colab.research.google.com/github/khaledrefai/face_recognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cmake

In [2]:
!pip install opencv-python

In [3]:
!pip install dlib

  Using cached https://files.pythonhosted.org/packages/35/8d/e4ddf60452e2fb1ce3164f774e68968b3f110f1cb4cd353235d56875799e/dlib-19.16.0.tar.gz
  Running setup.py bdist_wheel for dlib: started
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running.

In [4]:
!pip install face_recognition

  Using cached https://files.pythonhosted.org/packages/3f/ed/ad9a28042f373d4633fc8b49109b623597d6f193d3bbbef7780a5ee8eef2/face_recognition-1.2.3-py2.py3-none-any.whl


In [5]:
import numpy as np
import face_recognition
import cv2 as cv2 #OpenCV
from os import listdir
import time
import threading

In [6]:
facecascade = cv2.CascadeClassifier('d:/haarcascade_frontalface_default.xml')

In [8]:
import json

#put your employee pictures in this path as name_of_employee.jpg
employee_pictures = "d:/employeephotos/"

known_face_encodings =[]
known_face_names =[]

for file in listdir(employee_pictures):
    employee, extension = file.split(".")
    img = face_recognition.load_image_file('d:/employeephotos/%s.jpg' % (employee))
    #face_location = face_recognition.face_locations(img ,number_of_times_to_upsample=0, model="cnn")
    img_encoding = face_recognition.face_encodings(img)[0]
    known_face_encodings.append(img_encoding)
    known_face_names.append(employee)


print('employee representations retrieved successfully')      

employee representations retrieved successfully


In [9]:
def rescale_frame(frame, percent=75):
      width = int(frame.shape[1] * percent/ 100)
      height = int(frame.shape[0] * percent/ 100)
      dim = (width, height)
      return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA) 

In [12]:
import requests 
from IPython.core.display import display, HTML

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
video_capture = cv2.VideoCapture(0)
#video_capture = VideoCaptureAsync(0)
#video_capture.start()
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    #frame = rescale_frame(frame,50)
      
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]
    
    
    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame ,number_of_times_to_upsample=0, model="cnn")
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding,0.45)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
           

            
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
#video_capture.stop()
cv2.destroyAllWindows()